In [9]:
import os
import re
import glob
from pathlib import Path

import numpy as np
import pandas as pd
from datetime import datetime

import pygrib

In [40]:
def get_indexes_by_coord(coord, station_point, threshold=0.01):
    for (i, grid) in enumerate(coord):
        for (j, point) in enumerate(grid):
            if abs(point - station_point) <= threshold:
                return (i, j)
            
def update_message_value(grb, metar_value):
    lats, lons = grb.latlons()

    (i, i_j) = get_indexes_by_coord(lats, station_coords[0], threshold=1)
    (j_i, j) = get_indexes_by_coord(lons, station_coords[1], threshold=0.1)

    n_temp = np.array(grb.values, copy=True)
    n_temp[i, j] = metar_value
    grb['values'] = n_temp

    return grb.tostring()

def load_metar(grbs, filename):
    metar = pd.read_hdf(filename)
    
    surface_temp = grbs.select(name='Temperature')[41]
    surface_uwind = grbs.select(name='U component of wind')[41]
    surface_vwind = grbs.select(name='V component of wind')[41]
    surface_press = grbs.select(name='Surface pressure')[0]
    
    print(f"{surface_temp} \n{surface_uwind}, \n{surface_vwind}, \n{surface_press}")
    
    metar_data_by_messagenumber = {
        surface_temp.messagenumber: metar.temp,
        surface_uwind.messagenumber: metar.uwind,
        surface_vwind.messagenumber: metar.vwind,
        surface_press.messagenumber: metar.press
    }
    return metar_data_by_messagenumber

def save_grib(messages, filename):
    grbout = open(filename, 'wb')

    for msg in messages:
        grbout.write(msg)

    grbout.close()

In [41]:
grbs = pygrib.open('./gfs-data/gfs_20220331_12_00.pgrb2.0p25')

print(grbs.select(name='Temperature')[41].values[313, 1140])
print(grbs.select(name='U component of wind')[41].values[313, 1140])
print(grbs.select(name='V component of wind')[41].values[313, 1140])
print(grbs.select(name='Surface pressure')[0].values[313, 1140])

299.216748046875
-2.3136718750000003
-2.758544921875
100800.23125000001


In [42]:
pd.read_hdf('./ogimet-data/metar_20220331_12_00.h5')

,lat,long,temp,uwind,vwind,press
0,10.883333,-74.783333,299.15,-2.31475,-2.758611,100800.0


In [43]:
(station_lat, station_lon) = "10.883333,-74.783333".split(',')
station_coords = (float(station_lat), 359.75 + float(station_lon))

for metar_file in glob.glob('./ogimet-data/*.h5'):
    date = datetime.strptime(metar_file, './ogimet-data/metar_%Y%m%d_%H_00.h5')
    grib_file = './gfs-data/gfs_{0:%Y}{0:%m}{0:%d}_{0:%H}_00.pgrb2.0p25.*'.format(date)
    grib_file = glob.glob(grib_file)[0]

    print(f"Interpolate {grib_file}, based on {metar_file}")

    grbs = pygrib.open(grib_file)

    metar_data_by_messagenumber = load_metar(grbs, metar_file)

    messages = []
    for grb in grbs:
        if grb.messagenumber in metar_data_by_messagenumber.keys():
            messages.append(update_message_value(grb, metar_data_by_messagenumber[grb.messagenumber]))
        else:
            messages.append(grb.tostring())

    grbs.close()

    save_grib(messages, grib_file)

    print(f"{grib_file} saved in ./gfs-data")

Interpolate ./gfs-data/gfs_20220331_12_00.pgrb2.0p25, based on ./ogimet-data/metar_20220331_12_00.h5
563:Temperature:K (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 202203311200 
548:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100000 Pa:fcst time 0 hrs:from 202203311200, 
549:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100000 Pa:fcst time 0 hrs:from 202203311200, 
561:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 202203311200
./gfs-data/gfs_20220331_12_00.pgrb2.0p25 saved in ./gfs-data
Interpolate ./gfs-data/gfs_20220331_15_00.pgrb2.0p25, based on ./ogimet-data/metar_20220331_15_00.h5
563:Temperature:K (instant):regular_ll:surface:level 0:fcst time 3 hrs:from 202203311200 
548:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100000 Pa:fcst time 3 hrs:from 202203311200, 
549:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100000 Pa:fcst time 3 hrs:from 2

In [5]:
 pygrib.open('../gfs-data/gfs_20220401_12_00.pgrb2.0p25.*')

OSError: [Errno could not open %s] ../gfs-data/gfs_20220401_12_00.pgrb2.0p25.*

['../gfs-data/gfs_20220401_12_00.pgrb2.0p25.t006z']